In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from functions import *

In [6]:
def bureau_bb(bureau, bb):

    # Credit duration and credit/account end date difference
    bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
    bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
    
    # Credit to debt ratio and difference
    bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
    bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
    bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']
    bureau['BUREAU_CREDIT_FACT_DIFF'] = bureau['DAYS_CREDIT'] - bureau['DAYS_ENDDATE_FACT']
    bureau['BUREAU_CREDIT_ENDDATE_DIFF'] = bureau['DAYS_CREDIT'] - bureau['DAYS_CREDIT_ENDDATE']
    bureau['BUREAU_CREDIT_DEBT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']

    # CREDIT_DAY_OVERDUE :
    bureau['BUREAU_IS_DPD'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 0 else 0)
    bureau['BUREAU_IS_DPD_OVER120'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 120 else 0)

    bb, bb_cat = one_hot_encoder(bb, nan_as_category=True)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category=True)

    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size', 'mean']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']

    #Status of Credit Bureau loan during the month
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')

    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean', 'min'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean', 'max', 'sum'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean', 'sum'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
        'SK_ID_BUREAU': ['count'],
        'DAYS_ENDDATE_FACT': ['min', 'max', 'mean'],
        'ENDDATE_DIF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_FACT_DIFF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_ENDDATE_DIFF': ['min', 'max', 'mean'],
        'BUREAU_CREDIT_DEBT_RATIO': ['min', 'max', 'mean'],
        'DEBT_CREDIT_DIFF': ['min', 'max', 'mean'],
        'BUREAU_IS_DPD': ['mean', 'sum'],
        'BUREAU_IS_DPD_OVER120': ['mean', 'sum']
        }

    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')

    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')

    print('"Bureau/Bureau Balance" final shape:', bureau_agg.shape)
    return bureau_agg

# bureau

In [7]:
bureau = pd.read_csv('raw-data/dseb63_bureau.csv')
bureau.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR
0,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,254629
1,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,254629
2,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,254629
3,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,254629
4,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,254629


In [8]:
bb = pd.read_csv('raw-data/dseb63_bureau_balance.csv')
bb.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [9]:
bureau_agg = bureau_bb(bureau, bb)
bureau_agg.head(20)

"Bureau/Bureau Balance" final shape: (263491, 200)


,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,-63,-63,-63.000000,NaN,237.0,237.0,237.000000,-28.000000,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-2348,-2348,-2348.000000,NaN,-2044.0,-2044.0,-2044.000000,-18.000000,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2901,-30,-810.333333,1.222703e+06,-2597.0,274.0,-572.166667,-625.500000,0,0.0,...,0.0,0.0,0.0,20656.08,86445.0,38267.145,0.0,0.0,0.0,0.0
3,-2865,-116,-1131.428571,1.260530e+06,-2683.0,502.0,-658.714286,-805.000000,0,0.0,...,0.0,0.0,0.0,51705.00,90000.0,67921.875,0.0,0.0,0.0,0.0
4,-1056,-313,-773.333333,1.082075e+05,-509.0,1148.0,474.500000,-63.333333,0,0.0,...,0.0,0.0,0.0,112500.00,945000.0,450000.000,0.0,0.0,0.0,0.0
5,-2420,-67,-740.272727,5.312548e+05,-594.0,1759.0,810.000000,-234.363636,0,0.0,...,0.0,0.0,0.0,571500.00,11623500.0,4163454.750,0.0,0.0,0.0,0.0
6,-1883,-8,-638.714286,4.709609e+05,-733.0,956.0,234.571429,-125.142857,0,0.0,...,0.0,0.0,0.0,24210.00,1080000.0,552105.000,0.0,0.0,0.0,0.0
7,-2711,-2273,-2492.000000,9.592200e+04,-2345.0,-1543.0,-1944.000000,-1938.000000,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
8,-2675,-2538,-2606.500000,9.384500e+03,-2125.0,-711.0,-1418.000000,-1155.000000,0,0.0,...,0.0,0.0,0.0,157500.00,157500.0,157500.000,0.0,0.0,0.0,0.0


In [12]:
bureau_agg.isnull().sum().sort_values(ascending=False).head(10)

ACTIVE_ENDDATE_DIF_MEAN                262015
ACTIVE_ENDDATE_DIF_MAX                 262015
ACTIVE_ENDDATE_DIF_MIN                 262015
ACTIVE_DAYS_ENDDATE_FACT_MEAN          262006
ACTIVE_DAYS_ENDDATE_FACT_MAX           262006
ACTIVE_DAYS_ENDDATE_FACT_MIN           262006
ACTIVE_BUREAU_CREDIT_FACT_DIFF_MEAN    262006
ACTIVE_BUREAU_CREDIT_FACT_DIFF_MIN     262006
ACTIVE_BUREAU_CREDIT_FACT_DIFF_MAX     262006
ACTIVE_AMT_ANNUITY_MAX                 202145
dtype: int64

In [15]:
# Replace all inf values with nan values
bureau_agg.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill in missing values
imputer = SimpleImputer(strategy='median')
imputer.fit(bureau_agg)
bureau_agg = pd.DataFrame(imputer.transform(bureau_agg), columns=bureau_agg.columns, index=bureau_agg.index)

In [16]:
# export including index
bureau_agg.to_csv('processed-data/processed_bureau.csv', index=True)

In [17]:
bureau_agg = pd.read_csv('processed-data/processed_bureau.csv', index_col=0)
bureau_agg.head()

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,-63.0,-63.0,-63.000000,3.851066e+05,237.0,237.0,237.000000,-28.000000,0.0,0.0,...,0.0,0.0,0.0,44806.50,188082.0,117000.000,0.0,0.0,0.0,0.0
1,-2348.0,-2348.0,-2348.000000,3.851066e+05,-2044.0,-2044.0,-2044.000000,-18.000000,0.0,0.0,...,0.0,0.0,0.0,44806.50,188082.0,117000.000,0.0,0.0,0.0,0.0
2,-2901.0,-30.0,-810.333333,1.222703e+06,-2597.0,274.0,-572.166667,-625.500000,0.0,0.0,...,0.0,0.0,0.0,20656.08,86445.0,38267.145,0.0,0.0,0.0,0.0
3,-2865.0,-116.0,-1131.428571,1.260530e+06,-2683.0,502.0,-658.714286,-805.000000,0.0,0.0,...,0.0,0.0,0.0,51705.00,90000.0,67921.875,0.0,0.0,0.0,0.0
4,-1056.0,-313.0,-773.333333,1.082075e+05,-509.0,1148.0,474.500000,-63.333333,0.0,0.0,...,0.0,0.0,0.0,112500.00,945000.0,450000.000,0.0,0.0,0.0,0.0
